In [27]:
import ipywidgets as widgets
from IPython.display import display, Image, clear_output, HTML
import time
import random
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from jupyter_ui_poll import ui_events

In [28]:
#Functions
# data consent function
def consent():
    data_consent_info = """DATA CONSENT INFORMATION:
    
    Please read:
    
    we wish to record your response data
    to an anonymised public data repository.
    Your data will be used for educational teaching purposes
    practising data analysis and visualisation.
    
    Please type yes in the box below if you consent to the 
    upload."""
    
    print(data_consent_info)
    result = input("> ")
    if result == "yes":
        print("Thanks for your participation.")
        print("Please contact philip.lewis@ucl.ac.uk")
        print("If you have any questions or concerns")
        print("regarding the stored results.")

    else:
        # end code execution by raising an exception
        raise(Exception("User did not consent to continue test."))

# ID input function
def ID():
    id_instructions = """
    Enter your anonymised ID
    
    To generate an anonymous 4-letter unique user identifier please enter:
    - two letters based on the initials (first and last name) of a childhood friend
    - two letters based on the initials (first and last name) of a favourite actor / actress
    
    e.g. if your friend was called Charlie Brown and film star was Tom Cruise
    then your unique identifer would be CBTC
    """
    print(id_instructions)
    user_id = input("> ")
    print("User entered id:", user_id)
    return user_id

#`widget functions
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
    
# google form function
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [48]:
#the ANS test function
def ANS_test():
    
    # setting up for results and images
    results = {
        'user_id': [],
        'filename' :[],
        'nL': [],
        'nR': [],
        'ratio': [],
        'correct': [],
        'time': []
    }
    
    event_info = {
        'description': '',
        'time': -1
    }
    
    images = ["image1.png", "image2.png", "image3.png",  "image4.png"]
    answers = ["Right", "Right", "Left", "Right"]
    score = 0

    
    # consent statement
    consent()
    # ID input and appends results dictionary 
    user_id = ID()
    results['user_id'].append(user_id)

    # Intro to the test
    print("Welcome to the ANS test")
    time.sleep(1)
    
    print("You will have 3 seconds to select which side has more dots")
    time.sleep(1)

    # for loop to loop through the  images and collect answers for each
    for i in range(len(images)):
        pic = Image(images[i], width = 500)
        btn1 = widgets.Button(description="Left")
        btn2 = widgets.Button(description="Right")
        
        btn1.on_click(register_btn_event)
        btn2.on_click(register_btn_event)
        panel = widgets.HBox([btn1, btn2])
    
        time.sleep(1.5)
        display(pic)
        time.sleep(0.75)
        clear_output(wait=True)
    
        print("Which side had more dots?")
        display(panel)

        # Adds the time to the result dictionary
        result = wait_for_event(timeout=3)
        time_result = result['time']
        results['time'].append(time_result)
        

        # fill in dictionary with ratio and number of dots in each image
        if i == 0:
            results['nR'].append("12")
            results['nL'].append("9")
            results['ratio'].append("4:3")
            results['filename'].append("image1")
        elif i == 1:
            results['nR'].append("10")
            results['nL'].append("9")
            results['ratio'].append("10:9")
            results['filename'].append("image2")
        elif i == 2:
            results['nR'].append("12")
            results['nL'].append("14")
            results['ratio'].append("7:6")
            results['filename'].append("image3")
        elif i == 3:
            results['nR'].append("18")
            results['nL'].append("16")
            results['ratio'].append("9:8")
            results['filename'].append("image4")

        # checks the answer is correct and appends result dictionary 
        if result['description'] == answers[i]:
            print("Correct!")
            results['correct'].append("yes")
            score = score + 1
        elif result['description'] != answers[i]:
            print("Incorrect")
            results['correct'].append("no")
        elif result['description'] == '':
            print("No button was pressed! Answer more quickly")
            results['correct'].append("too slow")
            
        clear_output(wait=True)

    print(f"Thank you for taking part you scored {score} out of 4")
    # sends results to google form
    send_to_google_form(results, "https://docs.google.com/forms/d/e/1FAIpQLSd_dRCPzyqiFIrMa92ryC08POR6oNcnUUR3YWPYXgOmBz5YdA/viewform?usp=sf_link")


In [49]:
ANS_test()

Thank you for taking part you scored 4 out of 4
